In [1]:
# Data handling and Manipulation
import numpy as np
import pandas as pd

# Performance evaluation
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import \
                accuracy_score, \
                classification_report, \
                cohen_kappa_score, \
                matthews_corrcoef, \
                confusion_matrix, \
                roc_auc_score

# Model Saving
import pickle,os

#  Machine Learning Algorithm libraries

from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier
from catboost import CatBoostClassifier

In [2]:
def evaluate_performance(y_true,y_pred,pred_prob='NA'):
    cr = classification_report(y_true,y_pred)
    filtered_cr = [ line for line in cr.split('\n') if len(line)!=0]

    precision_0 = float(filtered_cr[1].split()[1])
    recall_0 = float(filtered_cr[1].split()[2])
    f1_0 = float(filtered_cr[1].split()[3])
    
    precision_1 = float(filtered_cr[2].split()[1])
    recall_1 = float(filtered_cr[2].split()[2])
    f1_1 = float(filtered_cr[2].split()[3])

    balanced_accuracy = (recall_0 + recall_1)/2
    acc_score = accuracy_score(y_true,y_pred)
    cohen_kappa = cohen_kappa_score(y_true,y_pred)
    matthews_corrcoef_score = matthews_corrcoef(y_true,y_pred)
    if pred_prob != 'NA':
        roc_auc = roc_auc_score(y_true,pred_prob)
    else:
        roc_auc = pred_prob
    

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    temp = dict(
        precision_0 = precision_0,
        precision_1 = precision_1,
        recall_0 = recall_0,
        recall_1 = recall_1,
        f1_0 = f1_0,
        f1_1 = f1_1,
        accuracy = acc_score,
        balanced_accuracy = balanced_accuracy,
        cohen_kappa = cohen_kappa,
        matthews_corrcoef_score = matthews_corrcoef_score,
        roc_auc_score = roc_auc,
        
        tn = tn,
        fp = fp,
        fn = fn,
        tp = tp
    )

    return temp


In [3]:
df = pd.read_csv('/path/to/fingerprint.csv')

seed = 42 # Set a seed for reproducibility (e.g., 42)
df = df.sample(frac=1, random_state = seed)

df.head()

,Name,APC2D1_C_C,APC2D1_C_N,APC2D1_C_O,APC2D1_C_S,APC2D1_C_P,APC2D1_C_F,APC2D1_C_Cl,APC2D1_C_Br,APC2D1_C_I,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
129252,0,23.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101853,0,18.0,6.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177265,0,13.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7323,0,20.0,8.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110451,0,13.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df['Name'].value_counts()

Name
0    175676
1      1675
Name: count, dtype: int64

# Train - Test split

In [5]:
train_df,test_df = train_test_split(df,test_size=0.2, stratify=df['Name'], random_state = 42 )

In [6]:
val_counts = train_df['Name'].value_counts()
val_counts

Name
0    140540
1      1340
Name: count, dtype: int64

# Majority - Minority Class splitting from Train set

In [7]:
majority_class_label =  val_counts.idxmax()
minority_class_label = val_counts.idxmin()

majority_class_df = train_df[train_df['Name'] == majority_class_label]
minority_class_df = train_df[train_df['Name'] == minority_class_label]

# Majority class Splitting 

In [8]:
# Number of subsets
n_subsets = 11

# Split the DataFrame
df_split = np.array_split(majority_class_df, n_subsets)

# Example: Accessing all subsets
for i, subset in enumerate(df_split, 1):
    print(f"Subset {i}: {len(subset)}")

C:\Users\ACDSD7\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Subset 1: 12777
Subset 2: 12777
Subset 3: 12777
Subset 4: 12777
Subset 5: 12776
Subset 6: 12776
Subset 7: 12776
Subset 8: 12776
Subset 9: 12776
Subset 10: 12776
Subset 11: 12776


## Creating Directory for saving Ensemble Models

In [9]:
cwd = os.getcwd()
os.makedirs('ENSEMBLE_MODELS',exist_ok=True)
path_to_ensemble_dir = os.path.join(cwd,'ENSEMBLE_MODELS')

### TRAINING ENSEMBLE  MODELS

In [10]:

##_-------------------------------------FOR_SAVING_THRESHOLDS_FOR_RESPECTIVE_MODELS-----------------------------------------------##
model_to_threshold_mapping = dict()

##----------------------------Setting-16000-inactive-and-1645-active-for-each-subset------------------------------------------------##

for i, subset in enumerate(df_split, 1):
    
    ##------------------------------------Pairing Majority subset class with minority class-----------------------------------##
    paired_dataset = pd.concat( [ subset, minority_class_df ], axis=0 )
    paired_dataset = paired_dataset.sample(frac=1, random_state=seed)
    
    X = paired_dataset.drop('Name', axis=1)
    y = paired_dataset['Name']
    
    ##--------------------------------------------------------Use-Classifier------------------------------------------------------##
    model = XGBClassifier(
        # **best_params
        random_state=42
    )
    model.fit(X,y)
    
    y_train_pred_proba = model.predict_proba(X)[:, minority_class_label]

    ##--------------------------------------------------------Use-Classifier------------------------------------------------------##
    
    mean_value = np.mean(y_train_pred_proba)
    model_name = 'xgb_'+ str(mean_value).split('.')[1]
    temp = dict(mean_value=mean_value)
    
    model_to_threshold_mapping[model_name] =  temp #mode_value

    path = os.path.join(path_to_ensemble_dir,f'{model_name}.pkl')
    with open(path, 'wb') as f:
        pickle.dump(model, f)
    print(f'XGB {i} ---> {mean_value} Model Name : {model_name}' )

XGB 1 ---> 0.09497527033090591 Model Name : xgb_09497527
XGB 2 ---> 0.09493295103311539 Model Name : xgb_09493295
XGB 3 ---> 0.09494435787200928 Model Name : xgb_09494436
XGB 4 ---> 0.09492084383964539 Model Name : xgb_094920844
XGB 5 ---> 0.09496831893920898 Model Name : xgb_09496832
XGB 6 ---> 0.09491197019815445 Model Name : xgb_09491197
XGB 7 ---> 0.09509076178073883 Model Name : xgb_09509076
XGB 8 ---> 0.09497106075286865 Model Name : xgb_09497106
XGB 9 ---> 0.09501765668392181 Model Name : xgb_09501766
XGB 10 ---> 0.09496447443962097 Model Name : xgb_094964474
XGB 11 ---> 0.09500715881586075 Model Name : xgb_09500716


666:	learn: 0.1689747	total: 8.61s	remaining: 4.3s
667:	learn: 0.1689194	total: 8.62s	remaining: 4.28s
668:	learn: 0.1688781	total: 8.63s	remaining: 4.27s
669:	learn: 0.1688064	total: 8.64s	remaining: 4.26s
670:	learn: 0.1687446	total: 8.66s	remaining: 4.25s
671:	learn: 0.1687097	total: 8.67s	remaining: 4.23s
672:	learn: 0.1685805	total: 8.68s	remaining: 4.22s
673:	learn: 0.1685681	total: 8.69s	remaining: 4.21s
674:	learn: 0.1684639	total: 8.71s	remaining: 4.2s
675:	learn: 0.1684061	total: 8.73s	remaining: 4.18s
676:	learn: 0.1683930	total: 8.74s	remaining: 4.17s
677:	learn: 0.1682804	total: 8.75s	remaining: 4.16s
678:	learn: 0.1682003	total: 8.77s	remaining: 4.14s
679:	learn: 0.1680512	total: 8.78s	remaining: 4.13s
680:	learn: 0.1679117	total: 8.79s	remaining: 4.12s
681:	learn: 0.1678601	total: 8.8s	remaining: 4.11s
682:	learn: 0.1678540	total: 8.82s	remaining: 4.09s
683:	learn: 0.1677479	total: 8.83s	remaining: 4.08s
684:	learn: 0.1676398	total: 8.84s	remaining: 4.07s
685:	learn: 0.1

831:	learn: 0.1565930	total: 10.7s	remaining: 2.17s
832:	learn: 0.1565333	total: 10.7s	remaining: 2.15s
833:	learn: 0.1565017	total: 10.8s	remaining: 2.14s
834:	learn: 0.1563510	total: 10.8s	remaining: 2.13s
835:	learn: 0.1563174	total: 10.8s	remaining: 2.12s
836:	learn: 0.1561686	total: 10.8s	remaining: 2.1s
837:	learn: 0.1561385	total: 10.8s	remaining: 2.09s
838:	learn: 0.1561124	total: 10.8s	remaining: 2.08s
839:	learn: 0.1560104	total: 10.8s	remaining: 2.06s
840:	learn: 0.1558695	total: 10.8s	remaining: 2.05s
841:	learn: 0.1557561	total: 10.9s	remaining: 2.04s
842:	learn: 0.1557461	total: 10.9s	remaining: 2.02s
843:	learn: 0.1556563	total: 10.9s	remaining: 2.01s
844:	learn: 0.1556227	total: 10.9s	remaining: 2s
845:	learn: 0.1555702	total: 10.9s	remaining: 1.99s
846:	learn: 0.1554926	total: 10.9s	remaining: 1.97s
847:	learn: 0.1554287	total: 10.9s	remaining: 1.96s
848:	learn: 0.1553370	total: 10.9s	remaining: 1.95s
849:	learn: 0.1553035	total: 11s	remaining: 1.93s
850:	learn: 0.1552

998:	learn: 0.1465133	total: 12.8s	remaining: 12.9ms
999:	learn: 0.1464220	total: 12.9s	remaining: 0us
Mean : 0.08534471344397423 	Median : 0.03763188388228554 	Mode : 0.01022473834708223
cat_3---> Learning rate set to 0.036391
0:	learn: 0.6580642	total: 11.9ms	remaining: 11.9s
1:	learn: 0.6254327	total: 23.5ms	remaining: 11.7s
2:	learn: 0.5964109	total: 35.4ms	remaining: 11.8s
3:	learn: 0.5697221	total: 47ms	remaining: 11.7s
4:	learn: 0.5451955	total: 59.4ms	remaining: 11.8s
5:	learn: 0.5230955	total: 71ms	remaining: 11.8s
6:	learn: 0.5027920	total: 83.1ms	remaining: 11.8s
7:	learn: 0.4831105	total: 96.1ms	remaining: 11.9s
8:	learn: 0.4654294	total: 108ms	remaining: 11.9s
9:	learn: 0.4499443	total: 120ms	remaining: 11.8s
10:	learn: 0.4362747	total: 131ms	remaining: 11.8s
11:	learn: 0.4227766	total: 143ms	remaining: 11.7s
12:	learn: 0.4103284	total: 155ms	remaining: 11.7s
13:	learn: 0.3999719	total: 168ms	remaining: 11.8s
14:	learn: 0.3900192	total: 179ms	remaining: 11.8s
15:	learn: 0.

167:	learn: 0.2267433	total: 2.14s	remaining: 10.6s
168:	learn: 0.2266333	total: 2.15s	remaining: 10.6s
169:	learn: 0.2263565	total: 2.17s	remaining: 10.6s
170:	learn: 0.2262891	total: 2.18s	remaining: 10.6s
171:	learn: 0.2261887	total: 2.19s	remaining: 10.5s
172:	learn: 0.2261307	total: 2.2s	remaining: 10.5s
173:	learn: 0.2259265	total: 2.21s	remaining: 10.5s
174:	learn: 0.2258978	total: 2.23s	remaining: 10.5s
175:	learn: 0.2257590	total: 2.24s	remaining: 10.5s
176:	learn: 0.2255992	total: 2.25s	remaining: 10.5s
177:	learn: 0.2255447	total: 2.27s	remaining: 10.5s
178:	learn: 0.2253644	total: 2.28s	remaining: 10.5s
179:	learn: 0.2252548	total: 2.29s	remaining: 10.4s
180:	learn: 0.2251611	total: 2.3s	remaining: 10.4s
181:	learn: 0.2249093	total: 2.32s	remaining: 10.4s
182:	learn: 0.2248508	total: 2.33s	remaining: 10.4s
183:	learn: 0.2246960	total: 2.34s	remaining: 10.4s
184:	learn: 0.2245969	total: 2.36s	remaining: 10.4s
185:	learn: 0.2245623	total: 2.37s	remaining: 10.4s
186:	learn: 0.

...

72:	learn: 0.0467223	total: 308ms	remaining: 3.91s
73:	learn: 0.0458976	total: 313ms	remaining: 3.92s
74:	learn: 0.0451297	total: 318ms	remaining: 3.92s
75:	learn: 0.0443424	total: 323ms	remaining: 3.92s
76:	learn: 0.0436093	total: 327ms	remaining: 3.91s
77:	learn: 0.0429285	total: 331ms	remaining: 3.91s
78:	learn: 0.0421669	total: 336ms	remaining: 3.91s
79:	learn: 0.0414504	total: 340ms	remaining: 3.91s
80:	learn: 0.0408150	total: 344ms	remaining: 3.91s
81:	learn: 0.0400109	total: 349ms	remaining: 3.9s
82:	learn: 0.0394119	total: 353ms	remaining: 3.9s
83:	learn: 0.0388687	total: 357ms	remaining: 3.89s
84:	learn: 0.0382290	total: 361ms	remaining: 3.89s
85:	learn: 0.0375579	total: 366ms	remaining: 3.89s
86:	learn: 0.0370150	total: 370ms	remaining: 3.88s
87:	learn: 0.0365524	total: 374ms	remaining: 3.88s
88:	learn: 0.0360708	total: 379ms	remaining: 3.88s
89:	learn: 0.0355244	total: 383ms	remaining: 3.87s
90:	learn: 0.0350021	total: 387ms	remaining: 3.87s
91:	learn: 0.0345728	total: 391ms

261:	learn: 0.0168667	total: 1.1s	remaining: 3.11s
262:	learn: 0.0168353	total: 1.11s	remaining: 3.1s
263:	learn: 0.0167839	total: 1.11s	remaining: 3.1s
264:	learn: 0.0166936	total: 1.12s	remaining: 3.1s
265:	learn: 0.0166606	total: 1.12s	remaining: 3.09s
266:	learn: 0.0166165	total: 1.13s	remaining: 3.09s
267:	learn: 0.0165914	total: 1.13s	remaining: 3.08s
268:	learn: 0.0165785	total: 1.13s	remaining: 3.08s
269:	learn: 0.0165686	total: 1.14s	remaining: 3.08s
270:	learn: 0.0165170	total: 1.14s	remaining: 3.07s
271:	learn: 0.0165071	total: 1.15s	remaining: 3.07s
272:	learn: 0.0164689	total: 1.15s	remaining: 3.06s
273:	learn: 0.0164365	total: 1.16s	remaining: 3.06s
274:	learn: 0.0164011	total: 1.16s	remaining: 3.06s
275:	learn: 0.0163140	total: 1.16s	remaining: 3.05s
276:	learn: 0.0162744	total: 1.17s	remaining: 3.05s
277:	learn: 0.0161641	total: 1.17s	remaining: 3.04s
278:	learn: 0.0161374	total: 1.18s	remaining: 3.04s
279:	learn: 0.0161017	total: 1.18s	remaining: 3.04s
280:	learn: 0.01

452:	learn: 0.0096150	total: 1.9s	remaining: 2.29s
453:	learn: 0.0095963	total: 1.9s	remaining: 2.29s
454:	learn: 0.0095845	total: 1.91s	remaining: 2.29s
455:	learn: 0.0095497	total: 1.91s	remaining: 2.28s
456:	learn: 0.0094959	total: 1.92s	remaining: 2.28s
457:	learn: 0.0094759	total: 1.92s	remaining: 2.27s
458:	learn: 0.0094571	total: 1.93s	remaining: 2.27s
459:	learn: 0.0094478	total: 1.93s	remaining: 2.27s
460:	learn: 0.0094057	total: 1.94s	remaining: 2.26s
461:	learn: 0.0093845	total: 1.94s	remaining: 2.26s
462:	learn: 0.0093681	total: 1.94s	remaining: 2.25s
463:	learn: 0.0093416	total: 1.95s	remaining: 2.25s
464:	learn: 0.0092862	total: 1.95s	remaining: 2.25s
465:	learn: 0.0092723	total: 1.96s	remaining: 2.24s
466:	learn: 0.0092405	total: 1.96s	remaining: 2.24s
467:	learn: 0.0092204	total: 1.97s	remaining: 2.23s
468:	learn: 0.0091676	total: 1.97s	remaining: 2.23s
469:	learn: 0.0091402	total: 1.97s	remaining: 2.23s
470:	learn: 0.0091171	total: 1.98s	remaining: 2.22s
471:	learn: 0.

644:	learn: 0.0051056	total: 2.7s	remaining: 1.49s
645:	learn: 0.0050740	total: 2.71s	remaining: 1.48s
646:	learn: 0.0050471	total: 2.71s	remaining: 1.48s
647:	learn: 0.0050366	total: 2.71s	remaining: 1.47s
648:	learn: 0.0050366	total: 2.72s	remaining: 1.47s
649:	learn: 0.0050238	total: 2.72s	remaining: 1.47s
650:	learn: 0.0049989	total: 2.73s	remaining: 1.46s
651:	learn: 0.0049716	total: 2.73s	remaining: 1.46s
652:	learn: 0.0049404	total: 2.74s	remaining: 1.45s
653:	learn: 0.0049301	total: 2.74s	remaining: 1.45s
654:	learn: 0.0049209	total: 2.75s	remaining: 1.45s
655:	learn: 0.0048943	total: 2.75s	remaining: 1.44s
656:	learn: 0.0048729	total: 2.75s	remaining: 1.44s
657:	learn: 0.0048654	total: 2.76s	remaining: 1.43s
658:	learn: 0.0048419	total: 2.76s	remaining: 1.43s
659:	learn: 0.0048267	total: 2.77s	remaining: 1.43s
660:	learn: 0.0048268	total: 2.77s	remaining: 1.42s
661:	learn: 0.0048164	total: 2.77s	remaining: 1.42s
662:	learn: 0.0048164	total: 2.78s	remaining: 1.41s
663:	learn: 0

842:	learn: 0.0029278	total: 3.51s	remaining: 653ms
843:	learn: 0.0029278	total: 3.51s	remaining: 649ms
844:	learn: 0.0029278	total: 3.52s	remaining: 645ms
845:	learn: 0.0029278	total: 3.52s	remaining: 641ms
846:	learn: 0.0029278	total: 3.52s	remaining: 637ms
847:	learn: 0.0029277	total: 3.53s	remaining: 632ms
848:	learn: 0.0029254	total: 3.53s	remaining: 628ms
849:	learn: 0.0029254	total: 3.54s	remaining: 624ms
850:	learn: 0.0029254	total: 3.54s	remaining: 620ms
851:	learn: 0.0029101	total: 3.54s	remaining: 616ms
852:	learn: 0.0029100	total: 3.55s	remaining: 611ms
853:	learn: 0.0029100	total: 3.55s	remaining: 607ms
854:	learn: 0.0029100	total: 3.56s	remaining: 603ms
855:	learn: 0.0029079	total: 3.56s	remaining: 599ms
856:	learn: 0.0029079	total: 3.56s	remaining: 595ms
857:	learn: 0.0029046	total: 3.57s	remaining: 591ms
858:	learn: 0.0029046	total: 3.57s	remaining: 586ms
859:	learn: 0.0029037	total: 3.58s	remaining: 582ms
860:	learn: 0.0029016	total: 3.58s	remaining: 578ms
861:	learn: 

# train and test splitting for prediction and evaluation

In [11]:
X_train, y_train = train_df.drop('Name',axis=1),train_df['Name']
X_test, y_test =  test_df.drop('Name',axis=1),test_df['Name']

print('X_train ---> active :',len([ label for label in y_train if label == 1]),end = '\t')
print('inactive :',len( [ label for label in y_train if label == 0] ))

print('X_test ---> active :',len([ label for label in y_test if label == 1]),end = '\t')
print('inactive :',len( [ label for label in y_test if label == 0] ))

X_train ---> active : 1340	inactive : 140540
X_test ---> active : 335	inactive : 35136


## **PERFORMANCE ON TRAIN SET**

In [12]:
train_pred_df = pd.DataFrame()
train_pred_df['y_true'] = y_train.tolist()

for model in os.listdir(path_to_ensemble_dir):
    if 'xgb' not in model:
        continue
    model_thres = model.split('.')[0].split('_')[1]
    model_thres = float(model_thres) / int('1'+('0' * len(model_thres)))

    model_path = os.path.join(path_to_ensemble_dir,model)
    # Load the model from the file
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
        
    pred_probs = loaded_model.predict_proba(X_train)[:,minority_class_label].tolist()
    pred_labels = [minority_class_label if v > model_thres else majority_class_label for v in  pred_probs]

    train_pred_df[model.split('.')[0]+'_pred_prob'] = pred_probs
    train_pred_df[model.split('.')[0]] = pred_labels

columns_to_check = [ c for c in list(train_pred_df.columns) if '_pred_prob' in c ]
train_pred_df['Final Pred Prob'] =  train_pred_df[columns_to_check].mean(axis=1)

columns_to_check = [ c for c in list(train_pred_df.columns) if 'xgb' in c and '_pred_prob' not in c]
train_pred_df['Final Prediction'] =  train_pred_df[columns_to_check].apply(lambda row: 1 if (row == 1).sum() > (row == 0).sum() else 0, axis=1)

train_pred_df.head()

,y_true,xgb_09491197_pred_prob,xgb_09491197,xgb_094920844_pred_prob,xgb_094920844,xgb_09493295_pred_prob,xgb_09493295,xgb_09494436_pred_prob,xgb_09494436,xgb_094964474_pred_prob,...,xgb_09497527_pred_prob,xgb_09497527,xgb_09500716_pred_prob,xgb_09500716,xgb_09501766_pred_prob,xgb_09501766,xgb_09509076_pred_prob,xgb_09509076,Final Pred Prob,Final Prediction
0,0,0.115939,1,0.197506,1,0.068613,0,0.064524,0,0.039133,...,0.034301,0,0.082676,0,0.007992,0,0.110979,1,0.070621,0
1,0,0.073765,0,0.349369,1,0.272308,1,0.026565,0,0.041200,...,0.029813,0,0.384493,1,0.278320,1,0.348787,1,0.206734,1
2,0,0.003535,0,0.023300,0,0.020511,0,0.013394,0,0.025731,...,0.043613,0,0.005653,0,0.033986,0,0.025670,0,0.021604,0
3,0,0.013701,0,0.003500,0,0.009292,0,0.004310,0,0.004729,...,0.007950,0,0.002747,0,0.014609,0,0.018083,0,0.008363,0
4,0,0.011308,0,0.008747,0,0.006610,0,0.017111,0,0.015640,...,0.006115,0,0.004853,0,0.010890,0,0.009810,0,0.009712,0


In [13]:
y_true = train_pred_df['y_true'].tolist()
y_pred = train_pred_df['Final Prediction'].tolist()
pred_prob = train_pred_df['Final Pred Prob'].tolist()
evaluate_performance(y_true,y_pred,pred_prob)

{'precision_0': 1.0,
 'precision_1': 0.08,
 'recall_0': 0.89,
 'recall_1': 0.99,
 'f1_0': 0.94,
 'f1_1': 0.14,
 'accuracy': 0.8887369608119537,
 'balanced_accuracy': 0.94,
 'cohen_kappa': 0.12875779120973851,
 'matthews_corrcoef_score': 0.2610676489727324,
 'roc_auc_score': 0.992730425183036,
 'tn': 124766,
 'fp': 15774,
 'fn': 12,
 'tp': 1328}

## **PERFORMANCE ON TEST SET**

In [14]:
test_pred_df = pd.DataFrame()
test_pred_df['y_true'] = y_test.tolist()

for model in os.listdir(path_to_ensemble_dir):
    if 'xgb' not in model:
        continue
    model_thres = model.split('.')[0].split('_')[1]
    model_thres = float(model_thres) / int('1'+('0' * len(model_thres)))

    model_path = os.path.join(path_to_ensemble_dir,model)
    # Load the model from the file
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
        
    pred_probs = loaded_model.predict_proba(X_test)[:,minority_class_label].tolist()
    pred_labels = [minority_class_label if v > model_thres else majority_class_label for v in  pred_probs]
    
    test_pred_df[model.split('.')[0]+'_pred_prob'] = pred_probs
    test_pred_df[model.split('.')[0]] = pred_labels

columns_to_check = [ c for c in list(test_pred_df.columns) if '_pred_prob' in c ]
test_pred_df['Final Pred Prob'] =  test_pred_df[columns_to_check].mean(axis=1)

columns_to_check = [ c for c in list(test_pred_df.columns) if 'xgb' in c and '_pred_prob' not in c]
test_pred_df['Final Prediction'] =  test_pred_df[columns_to_check].apply(lambda row: 1 if (row == 1).sum() > (row == 0).sum() else 0, axis=1)

test_pred_df.head()

,y_true,xgb_09491197_pred_prob,xgb_09491197,xgb_094920844_pred_prob,xgb_094920844,xgb_09493295_pred_prob,xgb_09493295,xgb_09494436_pred_prob,xgb_09494436,xgb_094964474_pred_prob,...,xgb_09497527_pred_prob,xgb_09497527,xgb_09500716_pred_prob,xgb_09500716,xgb_09501766_pred_prob,xgb_09501766,xgb_09509076_pred_prob,xgb_09509076,Final Pred Prob,Final Prediction
0,0,0.098869,1,0.033718,0,0.011201,0,0.010841,0,0.061060,...,0.024632,0,0.013135,0,0.019555,0,0.026266,0,0.030445,0
1,0,0.016131,0,0.001585,0,0.007490,0,0.003992,0,0.004085,...,0.011708,0,0.005031,0,0.003092,0,0.004008,0,0.005484,0
2,0,0.003768,0,0.000517,0,0.001139,0,0.000267,0,0.001009,...,0.006823,0,0.000273,0,0.006920,0,0.002705,0,0.002669,0
3,0,0.001948,0,0.085107,0,0.061891,0,0.021563,0,0.065548,...,0.026749,0,0.014401,0,0.015897,0,0.044293,0,0.037600,0
4,0,0.103617,1,0.078566,0,0.088583,0,0.346989,1,0.044948,...,0.440403,1,0.122600,1,0.157786,1,0.035037,0,0.142749,0


In [15]:
y_true = test_pred_df['y_true']
y_pred = test_pred_df['Final Prediction'].tolist()
pred_prob = test_pred_df['Final Pred Prob'].tolist()
evaluate_performance(y_true,y_pred,pred_prob)

{'precision_0': 1.0,
 'precision_1': 0.05,
 'recall_0': 0.88,
 'recall_1': 0.64,
 'f1_0': 0.93,
 'f1_1': 0.09,
 'accuracy': 0.8765470384257562,
 'balanced_accuracy': 0.76,
 'cohen_kappa': 0.073512678141189,
 'matthews_corrcoef_score': 0.1524936374654627,
 'roc_auc_score': 0.851540198597178,
 'tn': 30876,
 'fp': 4260,
 'fn': 119,
 'tp': 216}

## FOR EXTERNAL DATASET EVALUATION

In [16]:
ext_df = pd.read_csv('/path/to/fingerprint.csv')

Ext_X_test = ext_df.drop('Name',axis=1)
Ext_y_test = ext_df['Name']

ext_pred_df = pd.DataFrame()
ext_pred_df['y_true'] = Ext_y_test.tolist()

for model in os.listdir(path_to_ensemble_dir):
    if 'xgb' not in model:
        continue
    model_thres = model.split('.')[0].split('_')[1]
    model_thres = float(model_thres) / int('1'+('0' * len(model_thres)))

    model_path = os.path.join(path_to_ensemble_dir,model)
    # Load the model from the file
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
        
    pred_probs = loaded_model.predict_proba(Ext_X_test)[:,minority_class_label].tolist()
    pred_labels = [minority_class_label if v > model_thres else majority_class_label for v in  pred_probs]

    ext_pred_df[model.split('.')[0]+'_pred_prob'] = pred_probs
    ext_pred_df[model.split('.')[0]] = pred_labels

columns_to_check = [ c for c in list(ext_pred_df.columns) if '_pred_prob' in c ]
ext_pred_df['Final Pred Prob'] =  ext_pred_df[columns_to_check].mean(axis=1)

columns_to_check = [ c for c in list(ext_pred_df.columns) if 'xgb' in c and '_pred_prob' not in c]
ext_pred_df['Final Prediction'] =  ext_pred_df[columns_to_check].apply(lambda row: 1 if (row == 1).sum() > (row == 0).sum() else 0, axis=1)

ext_pred_df.head()

,y_true,xgb_09491197_pred_prob,xgb_09491197,xgb_094920844_pred_prob,xgb_094920844,xgb_09493295_pred_prob,xgb_09493295,xgb_09494436_pred_prob,xgb_09494436,xgb_094964474_pred_prob,...,xgb_09497527_pred_prob,xgb_09497527,xgb_09500716_pred_prob,xgb_09500716,xgb_09501766_pred_prob,xgb_09501766,xgb_09509076_pred_prob,xgb_09509076,Final Pred Prob,Final Prediction
0,0,0.189088,1,0.037583,0,0.732393,1,0.306964,1,0.184197,...,0.412895,1,0.069439,0,0.448120,1,0.011540,0,0.234394,1
1,0,0.002354,0,0.005193,0,0.001858,0,0.004948,0,0.002758,...,0.007721,0,0.002702,0,0.010366,0,0.002269,0,0.004528,0
2,0,0.034410,0,0.022305,0,0.015093,0,0.020381,0,0.023022,...,0.036816,0,0.042567,0,0.025972,0,0.021305,0,0.024755,0
3,0,0.011495,0,0.059807,0,0.013495,0,0.038627,0,0.150229,...,0.025582,0,0.002528,0,0.016055,0,0.003194,0,0.033461,0
4,0,0.007029,0,0.009324,0,0.007189,0,0.008122,0,0.010750,...,0.015035,0,0.016209,0,0.005383,0,0.008271,0,0.012256,0


In [17]:
y_true = ext_pred_df['y_true']
y_pred = ext_pred_df['Final Prediction'].tolist()
pred_prob = ext_pred_df['Final Pred Prob'].tolist()
evaluate_performance(y_true,y_pred,pred_prob)

{'precision_0': 0.99,
 'precision_1': 0.04,
 'recall_0': 0.77,
 'recall_1': 0.64,
 'f1_0': 0.87,
 'f1_1': 0.08,
 'accuracy': 0.7695547840294088,
 'balanced_accuracy': 0.7050000000000001,
 'cohen_kappa': 0.05258757032308381,
 'matthews_corrcoef_score': 0.12135552646935072,
 'roc_auc_score': 0.7722001277382381,
 'tn': 29750,
 'fp': 8807,
 'fn': 220,
 'tp': 395}